# Task1

### 1. ¿Qué es Programación Dinámica y cómo se relaciona con RL?

    Programación Dinámica (PD) es una técnica de optimización que resuelve problemas complejos dividiéndolos
    en subproblemas más simples y almacenando los resultados de estos subproblemas para evitar cálculos redundantes. 
    Fue desarrollada por Richard Bellman en la
    década de 1950 y se utiliza para resolver problemas de optimización que pueden descomponerse en etapas secuenciales
    
    En el contexto de Reinforcement Learning (RL), la programación dinámica se aplica en métodos como Iteración de
    valor e Iteración de Póliza. Estos métodos buscan encontrar la política óptima para un agente en un entorno de
    MDP (Markov Decision Process) utilizando principios de PD. RL puede ser visto como una extensión de PD donde 
    el entorno puede no ser completamente conocido de antemano, requiriendo que el agente explore y aprenda a partir 
    de la interacción con el entorno.


### 2. Explique en sus propias palabras el algoritmo de Iteración de Póliza.

    Iteración de Póliza es un algoritmo de PD utilizado en RL para encontrar la política óptima en un MDP. El proceso 
    alterna entre dos fases: evaluación de la póliza y mejora de la póliza.

    Evaluación de la Póliza: Dada una política (estrategia) inicial, se calcula el valor esperado de cada estado 
    siguiendo esa política. Esto implica resolver un sistema de ecuaciones para obtener los valores de estado.
    Mejora de la Póliza: Utilizando los valores de estado calculados, se actualiza la política para elegir las 
    acciones que maximicen el valor esperado en cada estado.
    El algoritmo repite estas fases hasta que la política deja de cambiar significativamente, lo que indica que
    se ha encontrado la política óptima.
    
### 3. Explique en sus propias palabras el algoritmo de Iteración de Valor

    Iteración de Valor es otro algoritmo de PD en RL que busca la política óptima iterando sobre los valores de 
    los estados directamente.
    Actualización de Valor: Se actualiza el valor de cada estado iterativamente utilizando la ecuación de Bellman. 
    Esta ecuación expresa el valor de un estado como el valor esperado de la recompensa inmediata más el valor 
    esperado del próximo estado, considerando todas las posibles acciones y transiciones.
    Derivación de la Póliza: Una vez que los valores de estado se estabilizan (convergen), se deriva la política
    óptima eligiendo en cada estado la acción que maximice el valor esperado del siguiente estado.
    El proceso de iteración de valor continúa hasta que los valores de estado convergen a los valores óptimos, 
    momento en el cual la política óptima puede ser extraída fácilmente.
 
### 4. En el laboratorio pasado,vimos que elvalor de los premios obtenidos se mantienen constantes, ¿porqué?

    El valor de los premios obtenidos se mantiene constante debido a la naturaleza de la función de recompensa en el 
    problema específico tratado. En muchos entornos de RL, las recompensas están definidas por una función fija que 
    no varía con el tiempo ni con las acciones del agente. Esto significa que, independientemente de las decisiones 
    que tome el agente, las recompensas asociadas a estados y transiciones particulares permanecen iguales.

    Esta constancia en las recompensas facilita el análisis y la solución de MDPs, ya que permite la aplicación directa
    de algoritmos de PD como la Iteración de Póliza e Iteración de Valor sin necesidad de ajustar las recompensas en 
    cada iteración.



### Referencias
    Sutton, R. S., & Barto, A. G. (2018). Reinforcement Learning: An Introduction.
    



In [1]:
import random

In [2]:
class MDP():
    def __init__(self):
        self.states = self.getStates()
        self.actions = self.getActions()
        self.map = self.getMap()
        self.transitions = self.getTransitions()
        self.rewards = self.getRewards()
        self.policy = self.getPolicy()
        self.V = {s: 0 for s in self.states}  # Inicializamos la función de valor en 0
        self.gamma = 0.9  # Factor de descuento

    def simulatePolicy(self, steps, initialState=0):
        rewardAcc = 0
        estado = initialState
        for _ in range(steps):
            action = self.policy[estado]
            sPrime = self.transitions[estado][action]
            reward = self.rewards[estado][action][sPrime]
            
            rewardAcc += reward
            estado = sPrime
            
            if estado == 5:  # Si el robot alcanza la meta (G), terminamos la simulación
                break
        return rewardAcc
    
    def getStates(self):
        states = [i for i in range(9)]
        return states

    def getMapSymbol(self, pos):
        return self.map[pos]

    def getRewards(self):
        rewards = {}
        for i in self.states:
            rewards[i] = {}
            for j in self.actions:
                sPrime = self.transitions[i][j]
                if self.getMapSymbol(sPrime) == 'G':
                    rewards[i][j] = {sPrime: 1}
                elif self.getMapSymbol(sPrime) == 'X':
                    rewards[i][j] = {sPrime: -1}
                else:
                    rewards[i][j] = {sPrime: 0}
        return rewards

    def getPolicy(self):
        policy = {}
        for i in self.states:
            policy[i] = random.choice(self.actions)
        return policy

    def checkPosition(self, state, action):
        if action == 'arriba':
            if (state - 3) < 0:
                return state
            else:
                return state - 3
        elif action == 'abajo':
            if (state + 3) > len(self.states) - 1:
                return state
            else:
                return state + 3
        elif action == 'izquierda':
            if (state % 3) == 0:
                return state
            else:
                return state - 1
        elif action == 'derecha':
            if (state % 3) == 2:
                return state
            else:
                return state + 1

    def getTransitions(self):
        p = {}
        for i in self.states:
            p[i] = {}
            for j in self.actions:
                p[i][j] = self.checkPosition(i, j)
        return p

    def getMap(self):
        map = ['S', ' ', 'X',
               ' ', 'X', ' ',
               ' ', ' ', 'G']
        return map

    def getActions(self):
        actions = ['arriba', 'abajo', 'izquierda', 'derecha']
        return actions

    def valueIteration(self, threshold=0.001):
        while True:
            delta = 0
            for s in self.states:
                v = self.V[s]
                max_value = float('-inf')
                for a in self.actions:
                    s_prime = self.transitions[s][a]
                    reward = self.rewards[s][a][s_prime]
                    value = reward + self.gamma * self.V[s_prime]
                    if value > max_value:
                        max_value = value
                self.V[s] = max_value
                delta = max(delta, abs(v - self.V[s]))
            if delta < threshold:
                break
        self.extractPolicy()

    def extractPolicy(self):
        policy = {}
        for s in self.states:
            max_value = float('-inf')
            best_action = None
            for a in self.actions:
                s_prime = self.transitions[s][a]
                reward = self.rewards[s][a][s_prime]
                value = reward + self.gamma * self.V[s_prime]
                if value > max_value:
                    max_value = value
                    best_action = a
            policy[s] = best_action
        self.policy = policy

    def policyIteration(self):
        stable = False
        while not stable:
            stable = True
            self.evaluatePolicy()
            for s in self.states:
                current_action = self.policy[s]
                max_value = float('-inf')
                best_action = None
                for a in self.actions:
                    s_prime = self.transitions[s][a]
                    reward = self.rewards[s][a][s_prime]
                    value = reward + self.gamma * self.V[s_prime]
                    if value > max_value:
                        max_value = value
                        best_action = a
                if current_action != best_action:
                    self.policy[s] = best_action
                    stable = False

    def evaluatePolicy(self, threshold=0.001):
        while True:
            delta = 0
            for s in self.states:
                v = self.V[s]
                a = self.policy[s]
                s_prime = self.transitions[s][a]
                reward = self.rewards[s][a][s_prime]
                self.V[s] = reward + self.gamma * self.V[s_prime]
                delta = max(delta, abs(v - self.V[s]))
            if delta < threshold:
                break

In [3]:
mdp = MDP()

print(mdp.policy)
# Iteración de valor
mdp.valueIteration()
print()
print("Función de valor óptimo (Iteración de valor):", mdp.V)
print()
print("Política óptima (Iteración de valor):", mdp.policy)

# Iteración de políticas
mdp.policyIteration()
print()
print("Función de valor óptimo (Iteración de políticas):", mdp.V)
print()
print("Política óptima (Iteración de políticas):", mdp.policy)

{0: 'arriba', 1: 'arriba', 2: 'arriba', 3: 'izquierda', 4: 'izquierda', 5: 'abajo', 6: 'izquierda', 7: 'abajo', 8: 'izquierda'}

Función de valor óptimo (Iteración de valor): {0: 7.281404955442832, 1: 7.091404955442831, 2: 8.991404955442832, 3: 8.091404955442831, 4: 8.991404955442832, 5: 9.991404955442832, 6: 8.991404955442832, 7: 9.991404955442832, 8: 9.991404955442832}

Política óptima (Iteración de valor): {0: 'abajo', 1: 'abajo', 2: 'abajo', 3: 'abajo', 4: 'abajo', 5: 'abajo', 6: 'derecha', 7: 'derecha', 8: 'abajo'}

Función de valor óptimo (Iteración de políticas): {0: 7.282264459898548, 1: 7.09226445989855, 2: 8.992264459898548, 3: 8.09226445989855, 4: 8.992264459898548, 5: 9.992264459898548, 6: 8.992264459898548, 7: 9.992264459898548, 8: 9.992264459898548}

Política óptima (Iteración de políticas): {0: 'abajo', 1: 'abajo', 2: 'abajo', 3: 'abajo', 4: 'abajo', 5: 'abajo', 6: 'derecha', 7: 'derecha', 8: 'abajo'}
